<a href="https://colab.research.google.com/github/savitskaya-om/python/blob/main/Untitled6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/datasciencedojo/datasets

Cloning into 'datasets'...
remote: Enumerating objects: 79, done.
remote: Total 79 (delta 0), reused 0 (delta 0), pack-reused 79
Unpacking objects: 100% (79/79), done.


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
RANDOM_SEED = 30

In [5]:
df = pd.read_csv ('datasets/titanic.csv')

In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
#import re
df["title"] = df.Name.str.extract(r"(,.+?\s)")
df["title"] = df["title"].str.replace(",\s", "")
df["title"] = df["title"].str.rstrip()
df["title"].value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


Mr.          517
Miss.        182
Mrs.         125
Master.       40
Dr.            7
Rev.           6
Mlle.          2
Major.         2
Col.           2
the            1
Capt.          1
Ms.            1
Sir.           1
Lady.          1
Mme.           1
Don.           1
Jonkheer.      1
Name: title, dtype: int64

In [8]:
df.loc[~df["title"].isin(["Mr.", "Miss.", "Mrs.", "Master."]),"title"] = "other"
df["title"].value_counts()

Mr.        517
Miss.      182
Mrs.       125
Master.     40
other       27
Name: title, dtype: int64

In [10]:
le = LabelEncoder()
le.fit(pd.unique(df["title"]))
df["title_encoded"] = le.transform(df["title"])

In [11]:
new_df = df[['Age','SibSp', 'Parch', 'Pclass', 'title_encoded']].dropna()
new_df.head()

,Age,SibSp,Parch,Pclass,title_encoded
0,22.0,1,0,3,2
1,38.0,1,0,1,3
2,26.0,0,0,3,1
3,35.0,1,0,1,3
4,35.0,0,0,3,2


In [12]:
new_df["Age_bin"] = pd.cut(new_df["Age"], bins = 5)

pd.unique(new_df["Age_bin"])

[(16.336, 32.252], (32.252, 48.168], (48.168, 64.084], (0.34, 16.336], (64.084, 80.0]]
Categories (5, interval[float64, right]): [(0.34, 16.336] < (16.336, 32.252] < (32.252, 48.168] <
                                           (48.168, 64.084] < (64.084, 80.0]]

In [13]:
le.fit(pd.unique(new_df["Age_bin"]))
#LabelEncoder()
new_df["Age_bin_encoded"] = le.transform(new_df["Age_bin"])

In [14]:
new_df.head()

,Age,SibSp,Parch,Pclass,title_encoded,Age_bin,Age_bin_encoded
0,22.0,1,0,3,2,"(16.336, 32.252]",1
1,38.0,1,0,1,3,"(32.252, 48.168]",2
2,26.0,0,0,3,1,"(16.336, 32.252]",1
3,35.0,1,0,1,3,"(32.252, 48.168]",2
4,35.0,0,0,3,2,"(32.252, 48.168]",2


In [15]:
X_train, X_test, y_train, y_test = train_test_split (new_df[['SibSp', 'Parch', 'Pclass', 'title_encoded']], new_df['Age_bin_encoded'], test_size = 0.3, random_state = RANDOM_SEED)

In [16]:
X_train.shape, X_test.shape

((499, 4), (215, 4))

In [17]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [18]:
pred = model.predict(X_test)
pred = le.inverse_transform(pred)

In [19]:
df_w_predict = X_test.copy()
df_w_predict = df_w_predict.join(new_df["Age"])
df_w_predict["real_age_bin"] = le.inverse_transform(y_test) 
df_w_predict["age_bin_encoded"] = y_test
df_w_predict["predict"] = pred
df_w_predict.head()

,SibSp,Parch,Pclass,title_encoded,Age,real_age_bin,age_bin_encoded,predict
349,0,0,3,2,42.0,"(32.252, 48.168]",2,"(16.336, 32.252]"
808,0,0,2,2,39.0,"(32.252, 48.168]",2,"(16.336, 32.252]"
380,0,0,1,1,42.0,"(32.252, 48.168]",2,"(16.336, 32.252]"
845,0,0,3,2,42.0,"(32.252, 48.168]",2,"(16.336, 32.252]"
96,0,0,1,2,71.0,"(64.084, 80.0]",4,"(32.252, 48.168]"


In [20]:
df_w_predict["dif"] = [df_w_predict.real_age_bin.iloc[i] == df_w_predict.predict.iloc[i] for i in range(len(df_w_predict))]
print(f"Модель предсказала правильный интервал для возраста пассажира в {round (df_w_predict.dif.sum()/df_w_predict.dif.count()*100,2)}% случаев")

Модель предсказала правильный интервал для возраста пассажира в 55.81% случаев


In [26]:
from sklearn.metrics import f1_score
print('F1 score', f1_score(y_test,model.predict(X_test), average = 'micro'))

F1 score 0.5581395348837209


In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(df_w_predict["age_bin_encoded"], model.predict(X_test))

0.5302325581395348

In [27]:
frame = {'Важность параметра':model.feature_importances_, 'Фича':X_train.columns}
result = pd.DataFrame(frame)
result.sort_values(by='Важность параметра', ascending=False)

,Важность параметра,Фича
3,0.462489,title_encoded
1,0.222735,Parch
2,0.176702,Pclass
0,0.138074,SibSp


In [ ]:
df_w_predict.head()

,SibSp,Parch,Pclass,title_encoded,real_age_bin,Age,predict,dif,rand_age,dif_rand,rand_age_bin
349,0,0,3,2,"(32.252, 48.168]",42.0,"(16.336, 32.252]",False,4,False,"(64.084, 80.0]"
808,0,0,2,2,"(32.252, 48.168]",39.0,"(16.336, 32.252]",False,4,False,"(64.084, 80.0]"
380,0,0,1,1,"(32.252, 48.168]",42.0,"(16.336, 32.252]",False,3,False,"(48.168, 64.084]"
845,0,0,3,2,"(32.252, 48.168]",42.0,"(16.336, 32.252]",False,0,False,"(0.34, 16.336]"
96,0,0,1,2,"(64.084, 80.0]",71.0,"(32.252, 48.168]",False,2,False,"(32.252, 48.168]"


А какой будет точность пресказания, если возратсную категорию будем выбирать случайным образом?

In [ ]:
np.random.seed(RANDOM_SEED)
df_w_predict["rand_age"] = np.random.choice(pd.unique(new_df["Age_bin_encoded"]), size = len(X_test))
df_w_predict["rand_age_bin"] = le.inverse_transform(df_w_predict["rand_age"]) 
df_w_predict["dif_rand"] = [df_w_predict.real_age_bin.iloc[i] == df_w_predict.rand_age_bin.iloc[i] for i in range(len(df_w_predict))]
print(f"Модель предсказала правильный интервал для возраста пассажира в {round (df_w_predict.dif_rand.sum()/df_w_predict.dif.count()*100,2)}% случаев")

Модель предсказала правильный интервал для возраста пассажира в 16.28% случаев


**Попробуем другой способ разбития на возрастные категории**

In [ ]:
new_df_2 = df[['Age','SibSp', 'Parch', 'Pclass', 'title_encoded']].dropna()
new_df_2.head()

,Age,SibSp,Parch,Pclass,title_encoded
0,22.0,1,0,3,2
1,38.0,1,0,1,3
2,26.0,0,0,3,1
3,35.0,1,0,1,3
4,35.0,0,0,3,2


In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
disc = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy = 'kmeans')
test_disc = disc.fit_transform(new_df_2)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_discretization.py:219: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  centers = km.fit(column[:, None]).cluster_centers_[:, 0]
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_discretization.py:233: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 3 are removed. Consider decreasing the number of bins.
  "decreasing the number of bins." % jj


In [ ]:
new_df_2["Age_binned"] = disc.fit_transform(np.array(new_df_2.Age).reshape(-1,1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split (new_df_2[['SibSp', 'Parch', 'Pclass', 'title_encoded']], new_df_2['Age_binned'], test_size = 0.3, random_state = RANDOM_SEED)

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred = model.predict(X_test)

In [ ]:
df_w_predict = X_test.copy()
df_w_predict["real_age_bin"] = y_test
df_w_predict["predict"] = pred
df_w_predict.head()

,SibSp,Parch,Pclass,title_encoded,real_age_bin,predict
349,0,0,3,2,3.0,1.0
808,0,0,2,2,2.0,1.0
380,0,0,1,1,3.0,2.0
845,0,0,3,2,3.0,1.0
96,0,0,1,2,4.0,2.0


In [ ]:
df_w_predict["dif"] = [df_w_predict.real_age_bin.iloc[i] == df_w_predict.predict.iloc[i] for i in range(len(df_w_predict))]
print(f"Модель предсказала правильный интервал для возраста пассажира в {round (df_w_predict.dif.sum()/df_w_predict.dif.count()*100,2)}% случаев")

Модель предсказала правильный интервал для возраста пассажира в 41.4% случаев


In [ ]:
mean_absolute_error(df_w_predict["real_age_bin"], df_w_predict["predict"])

0.786046511627907